<a href="https://colab.research.google.com/github/sanchit2843/SelfDrivingCar/blob/master/Trafficsignclassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/traffic-signs-data.zip

--2019-07-01 08:10:38--  https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/traffic-signs-data.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.116.33
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.116.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123524425 (118M) [application/zip]
Saving to: ‘traffic-signs-data.zip’

traffic-signs-data. 100%[===================>] 117.80M  36.4MB/s    in 3.2s    

2019-07-01 08:10:41 (36.4 MB/s) - ‘traffic-signs-data.zip’ saved [123524425/123524425]



In [0]:
!unzip traffic-signs-data.zip

Archive:  traffic-signs-data.zip
  inflating: test.p                  
  inflating: train.p                 
  inflating: valid.p                 


In [0]:
import pickle
train = pickle.load( open( "train.p", "rb" ) )
test = pickle.load( open( "test.p", "rb" ) )
valid = pickle.load( open( "valid.p", "rb" ) )

In [0]:
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,StandardScaler
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.callbacks import ReduceLROnPlateau
from keras.layers import Flatten,Dropout
from keras.layers import BatchNormalization,Average
import collections

In [0]:
onehotencoder = OneHotEncoder(categorical_features=[0])
y_train = np.reshape(y_train,(-1,1))
y_train = onehotencoder.fit_transform(y_train).toarray()
y_valid = np.reshape(y_valid,(-1,1))
y_valid = onehotencoder.fit_transform(y_valid).toarray()
y_test = np.reshape(y_test,(-1,1))
y_test = onehotencoder.fit_transform(y_test).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data w

In [0]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2
    )
test_datagen = ImageDataGenerator(
    featurewise_std_normalization=True,
    )

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:348: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [0]:
classifier = Sequential()
classifier.add(Conv2D(64,5,5,input_shape = (32,32,3),activation = 'elu'))
classifier.add(BatchNormalization())
classifier.add(BatchNormalization())
classifier.add(Conv2D(128,5,5,activation = 'elu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Dropout(0.2))
classifier.add(Conv2D(256,5,5,activation = 'elu'))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.25))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(BatchNormalization())
classifier.add(Dropout(0.4))
classifier.add(Flatten())
classifier.add(Dense(output_dim = 1000, init = 'uniform' , activation ='elu',input_dim = 784))
#classifier.add(Dense(output_dim = 1000, init = 'uniform' , activation ='relu'))
classifier.add(Dense(output_dim = 43, init = 'uniform' , activation ='softmax'))

#compiling model
classifier.compile(optimizer = 'adam',loss = 'categorical_crossentropy' , metrics = ['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1,patience=1, min_lr=0.0001)
callbacks_list = [reduce_lr]
#fitting cnn to training set
classifier.fit_generator(train_datagen.flow(X_train, y_train, batch_size=300),steps_per_epoch=len(X_train) / 300,validation_data=test_datagen.flow(X_valid,y_valid,batch_size = 300), validation_steps=len(X_valid)/300,nb_epoch = 20 ,callbacks = callbacks_list)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (5, 5), input_shape=(32, 32, 3..., activation="elu")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), activation="elu")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (5, 5), activation="elu")`
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="elu", input_dim=784, units=1000, kernel_initializer="uniform")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=43, kernel_initializer="uniform")`
/usr/local/lib/python3.6/dist-packages/ipyker

Epoch 1/40


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


116/115 [==============================] - 25s 219ms/step - loss: 2.2916 - acc: 0.4205 - val_loss: 2.9092 - val_acc: 0.4231
Epoch 2/40
116/115 [==============================] - 22s 191ms/step - loss: 0.8481 - acc: 0.7352 - val_loss: 1.4906 - val_acc: 0.6175
Epoch 3/40
116/115 [==============================] - 23s 195ms/step - loss: 0.3727 - acc: 0.8797 - val_loss: 1.0403 - val_acc: 0.7528
Epoch 4/40
116/115 [==============================] - 23s 196ms/step - loss: 0.2281 - acc: 0.9270 - val_loss: 0.6947 - val_acc: 0.8197
Epoch 5/40
116/115 [==============================] - 22s 193ms/step - loss: 0.1568 - acc: 0.9506 - val_loss: 0.4712 - val_acc: 0.9002
Epoch 6/40
116/115 [==============================] - 23s 196ms/step - loss: 0.1234 - acc: 0.9606 - val_loss: 0.4190 - val_acc: 0.9050
Epoch 7/40
116/115 [==============================] - 22s 193ms/step - loss: 0.0928 - acc: 0.9708 - val_loss: 0.4179 - val_acc: 0.9136
Epoch 8/40
116/115 [==============================] - 24s 204ms/st

In [0]:
scores = classifier.evaluate(X_test,y_test)

12630/12630 [==============================] - 4s 285us/step


In [0]:
print(scores)

[0.23972550659421193, 0.9538400633129352]


In [0]:
classifier.save("trafficsignclassifier.h5")